### in this example, we try to intergrate another LLM that call langda as a module to solve crusial problems

principle:
- Question + langda code with "/* CONTENT */" placeholder only => LLM_MAIN: decide what to fill in each placeholder
- fill the content => langda_solve => complete code => result
- LLM_MAIN answer the Question with the result

In [ ]:
from pydantic import BaseModel, Field
from pydantic_settings import BaseSettings, SettingsConfigDict
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from langda import langda_solve, invoke_agent
from dotenv import find_dotenv

task_solve_template = """
% about burglary
langda(LLM:"/* BURGLARY */").
% about earthquake
langda(LLM:"/* EARTHQUAKE */").

% about case 1
langda(LLM:"/* ALARM1 */",FUP:"false").
% about case 2
langda(LLM:"/* ALARM2 */",FUP:"false").

% law for alarm
alarm :- langda(LLM:"/* CONTENT3 */",FUP:"false").
query(alarm).
"""

task_solve_template2 = """
% about burglary

% about earthquake

% about case 1

% about case 2

% law for alarm

query(alarm).
"""

class LLMConfig(BaseModel):
    model: str
    api_key: str
    api_typ: str = "Bearer"
    api_ver: str = "2024-01-01"
    temperature: float = 0.2

class AgentSettings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=find_dotenv(),
        env_file_encoding="utf-8",
        env_nested_delimiter="_",
        env_nested_max_split=1,
        extra="ignore"
    )
    deepseek: LLMConfig
    openai: LLMConfig
    groq: LLMConfig

class LangdaAgentExecutor(BaseModel):
    cfgs: AgentSettings = Field(default_factory=AgentSettings)    

    def solve(self, question) -> str:
        system_prompt1 = f"""You have following template to solve the tasks:
<Template>
{task_solve_template}
</Template>
please think step by step, and give content to each placeholder /* CONTENT */, which could save the question. 
</Instruction>
<Final_Answer>
Please generate content in following form for each placeholder:

```content
what the logic is about content 1
```
```content
what the logic is about content 2
```

If there's no content needed for certain placeholder:
```content
//nothing
```

"""
        system_prompt2 = f"""You have following template to solve the tasks:
<Template>
{task_solve_template2}
</Template>
please think step by step, and give the full problog code that could solve the task
</Instruction>


"""

        chatprompt_template = ChatPromptTemplate.from_messages([
            ("system", system_prompt2),
            ("human", question)
        ])

        llm = ChatDeepSeek(
            model=self.cfgs.deepseek.model,
            temperature=self.cfgs.deepseek.temperature,
            api_key=self.cfgs.deepseek.api_key,
        )
        
        chain = chatprompt_template | llm | StrOutputParser()
        
        result = chain.invoke({})
        return result



In [18]:
question = """
The probability of a burglary is 0.8, and the probability of an earthquake is 0.2. 
If we have the following situations:
 1. When an earthquake occurs but no burglary occurs, an alarm is sounded
 2. When both a burglary and an earthquake occur, an alarm is sounded
What is the probability of an alarm being sounded?
"""

executor = LangdaAgentExecutor()
result = executor.solve(question)
print(result)

To solve this problem, we need to model the given probabilities and conditions in ProbLog. Here's the step-by-step solution:

1. Define the probabilities of burglary and earthquake:
   - burglary with probability 0.8
   - earthquake with probability 0.2

2. Define the cases when the alarm sounds:
   - Case 1: earthquake occurs and no burglary occurs
   - Case 2: both burglary and earthquake occur

3. The alarm sounds if either Case 1 or Case 2 occurs

Here's the complete ProbLog code:

```prolog
% about burglary
0.8::burglary.

% about earthquake
0.2::earthquake.

% about case 1 (earthquake and no burglary)
case1 :- earthquake, \+ burglary.

% about case 2 (both burglary and earthquake)
case2 :- burglary, earthquake.

% law for alarm
alarm :- case1.
alarm :- case2.

query(alarm).
```

This code models:
- The independent probabilities of burglary (0.8) and earthquake (0.2)
- The two cases when the alarm sounds:
  1. When there's an earthquake but no burglary (case1)
  2. When both burgl

In [ ]:
def extract_content_blocks(text: str) -> List[str]:
    pattern = r'```content\s*\n(.*?)\n```'
    matches = re.findall(pattern, text, re.DOTALL)
    content_blocks = [match.strip() for match in matches]
    return content_blocks